In [18]:
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

## Random Forest Model used on Bollinger,SMA,and EMA signals

In [19]:
#read in sma data
sma_forex = Path("sma_forex.csv")
sma = pd.read_csv(sma_forex)
sma.set_index(sma['DateTime'], inplace=True)
sma = sma.drop(columns= "DateTime")
sma.head()

,Close,SMA5,SMA15,Signal,Entry/Exit
DateTime,,,,,
2020-02-02 17:59:00,1.10846,1.108636,1.108849,0.0,0.0
2020-02-02 18:00:00,1.10878,1.108637,1.108838,0.0,0.0
2020-02-02 18:01:00,1.10882,1.108643,1.108832,0.0,0.0
2020-02-02 18:02:00,1.10887,1.108655,1.108826,0.0,0.0
2020-02-02 18:03:00,1.10898,1.108677,1.108823,0.0,0.0


In [20]:
#Read in EMA data
EMA_forex = Path("EMA_forex.csv")
ema = pd.read_csv(EMA_forex)
ema.set_index(ema['DateTime'], inplace=True)
ema = ema.drop(columns= "DateTime")
ema.head()

,Open,High,Low,Close,fast_close,slow_close,crossover_long,crossover_short,crossover_signal
DateTime,,,,,,,,,
2020-02-02 17:00:00,1.10939,1.10939,1.10939,1.10939,1.109390,1.109390,0.0,0.0,0.0
2020-02-02 17:01:00,1.10934,1.10934,1.10921,1.10921,1.109294,1.109298,0.0,-1.0,-1.0
2020-02-02 17:02:00,1.10922,1.10922,1.10922,1.10922,1.109266,1.109271,0.0,-1.0,-1.0
2020-02-02 17:03:00,1.10923,1.10923,1.10918,1.10918,1.109240,1.109247,0.0,-1.0,-1.0
2020-02-02 17:04:00,1.10921,1.10921,1.10921,1.10921,1.109232,1.109239,0.0,-1.0,-1.0


In [21]:
#read in Bollinger data
bollinger_csv = Path("../Travis/Bollinger.csv")
bol = pd.read_csv(bollinger_csv)
bol.set_index(bol['DateTime'], inplace=True)
bol = bol.drop(columns= "DateTime")
bol.head()

,Open,High,Low,Close,Volume,return,bollinger_mid_band,bollinger_std,bollinger_upper_band,bollinger_lower_band,bollinger_long,bollinger_short,bollinger_signal
DateTime,,,,,,,,,,,,,
2020-02-02 17:19:00,1.10911,1.10913,1.10910,1.10910,0.0,-0.000009,1.109183,0.000061,1.109244,1.109123,1.0,0.0,1.0
2020-02-02 17:20:00,1.10909,1.10912,1.10909,1.10911,0.0,0.000009,1.109169,0.000039,1.109208,1.109131,1.0,0.0,1.0
2020-02-02 17:21:00,1.10912,1.10913,1.10909,1.10910,0.0,-0.000009,1.109164,0.000041,1.109205,1.109123,1.0,0.0,1.0
2020-02-02 17:22:00,1.10911,1.10912,1.10877,1.10883,0.0,-0.000243,1.109145,0.000083,1.109228,1.109061,1.0,0.0,1.0
2020-02-02 17:23:00,1.10884,1.10885,1.10884,1.10885,0.0,0.000018,1.109128,0.000106,1.109234,1.109022,1.0,0.0,1.0


In [22]:
#Concat all three dfs into one name data frame rfm (random forest model)
rfm = pd.concat([bol, ema, sma], axis="columns", join="inner")
rfm.dropna(inplace=True)

#Change index to datetime
rfm.index = pd.to_datetime(rfm.index)

rfm.head()

,Open,High,Low,Close,Volume,return,bollinger_mid_band,bollinger_std,bollinger_upper_band,bollinger_lower_band,...,fast_close,slow_close,crossover_long,crossover_short,crossover_signal,Close,SMA5,SMA15,Signal,Entry/Exit
DateTime,,,,,,,,,,,,,,,,,,,,,
2020-02-02 17:59:00,1.10849,1.10853,1.10844,1.10846,0.0,-0.000027,1.108629,0.000174,1.108803,1.108455,...,1.108613,1.108726,0.0,-1.0,-1.0,1.10846,1.108636,1.108849,0.0,0.0
2020-02-02 18:00:00,1.10851,1.10888,1.10851,1.10878,0.0,0.000289,1.108653,0.000160,1.108812,1.108493,...,1.108635,1.108728,0.0,-1.0,-1.0,1.10878,1.108637,1.108838,0.0,0.0
2020-02-02 18:01:00,1.10880,1.10890,1.10877,1.10882,0.0,0.000036,1.108682,0.000129,1.108811,1.108553,...,1.108659,1.108732,0.0,-1.0,-1.0,1.10882,1.108643,1.108832,0.0,0.0
2020-02-02 18:02:00,1.10882,1.10891,1.10881,1.10887,0.0,0.000045,1.108698,0.000132,1.108830,1.108565,...,1.108686,1.108737,0.0,-1.0,-1.0,1.10887,1.108655,1.108826,0.0,0.0
2020-02-02 18:03:00,1.10888,1.10898,1.10888,1.10898,0.0,0.000099,1.108714,0.000146,1.108860,1.108567,...,1.108724,1.108746,0.0,-1.0,-1.0,1.10898,1.108677,1.108823,0.0,0.0


In [23]:
#drop duplicated "Close" columns
rfm = rfm.drop(rfm.columns[16], axis=1)
rfm.head()

,Open,High,Low,Volume,return,bollinger_mid_band,bollinger_std,bollinger_upper_band,bollinger_lower_band,bollinger_long,...,Low,fast_close,slow_close,crossover_long,crossover_short,crossover_signal,SMA5,SMA15,Signal,Entry/Exit
DateTime,,,,,,,,,,,,,,,,,,,,,
2020-02-02 17:59:00,1.10849,1.10853,1.10844,0.0,-0.000027,1.108629,0.000174,1.108803,1.108455,0.0,...,1.10844,1.108613,1.108726,0.0,-1.0,-1.0,1.108636,1.108849,0.0,0.0
2020-02-02 18:00:00,1.10851,1.10888,1.10851,0.0,0.000289,1.108653,0.000160,1.108812,1.108493,0.0,...,1.10851,1.108635,1.108728,0.0,-1.0,-1.0,1.108637,1.108838,0.0,0.0
2020-02-02 18:01:00,1.10880,1.10890,1.10877,0.0,0.000036,1.108682,0.000129,1.108811,1.108553,0.0,...,1.10877,1.108659,1.108732,0.0,-1.0,-1.0,1.108643,1.108832,0.0,0.0
2020-02-02 18:02:00,1.10882,1.10891,1.10881,0.0,0.000045,1.108698,0.000132,1.108830,1.108565,0.0,...,1.10881,1.108686,1.108737,0.0,-1.0,-1.0,1.108655,1.108826,0.0,0.0
2020-02-02 18:03:00,1.10888,1.10898,1.10888,0.0,0.000099,1.108714,0.000146,1.108860,1.108567,0.0,...,1.10888,1.108724,1.108746,0.0,-1.0,-1.0,1.108677,1.108823,0.0,0.0


In [42]:
# Set x variable list of features
x_var_list = ['crossover_signal', 'bollinger_signal']

# Filter by x-variable list
rfm[x_var_list].head()

,crossover_signal,bollinger_signal
DateTime,,
2020-02-02 18:00:00,-1.0,0.0
2020-02-02 18:01:00,-1.0,0.0
2020-02-02 18:02:00,-1.0,-1.0
2020-02-02 18:03:00,-1.0,-1.0
2020-02-02 18:04:00,-1.0,-1.0


In [43]:
# Shift DataFrame values by 1
rfm[x_var_list] = rfm[x_var_list].shift(1)

rfm[x_var_list].head()

,crossover_signal,bollinger_signal
DateTime,,
2020-02-02 18:00:00,NaN,NaN
2020-02-02 18:01:00,-1.0,0.0
2020-02-02 18:02:00,-1.0,0.0
2020-02-02 18:03:00,-1.0,-1.0
2020-02-02 18:04:00,-1.0,-1.0


In [44]:
# Drop NAs and replace positive/negative infinity values
rfm.dropna(subset=x_var_list, inplace=True)
rfm.dropna(subset=['return'], inplace=True)
rfm = rfm.replace([np.inf, -np.inf], np.nan)
rfm.head()

,Open,High,Low,Volume,return,bollinger_mid_band,bollinger_std,bollinger_upper_band,bollinger_lower_band,bollinger_long,...,slow_close,crossover_long,crossover_short,crossover_signal,SMA5,SMA15,Signal,Entry/Exit,positive_return,negitive_returns
DateTime,,,,,,,,,,,,,,,,,,,,,
2020-02-02 18:01:00,1.10880,1.10890,1.10877,0.0,0.000036,1.108682,0.000129,1.108811,1.108553,0.0,...,1.108732,0.0,-1.0,-1.0,1.108643,1.108832,0.0,0.0,1.0,0.0
2020-02-02 18:02:00,1.10882,1.10891,1.10881,0.0,0.000045,1.108698,0.000132,1.108830,1.108565,0.0,...,1.108737,0.0,-1.0,-1.0,1.108655,1.108826,0.0,0.0,1.0,0.0
2020-02-02 18:03:00,1.10888,1.10898,1.10888,0.0,0.000099,1.108714,0.000146,1.108860,1.108567,0.0,...,1.108746,0.0,-1.0,-1.0,1.108677,1.108823,0.0,0.0,1.0,0.0
2020-02-02 18:04:00,1.10898,1.10902,1.10895,0.0,0.000027,1.108731,0.000160,1.108891,1.108571,0.0,...,1.108756,1.0,0.0,-1.0,1.108704,1.108819,0.0,0.0,1.0,0.0
2020-02-02 18:05:00,1.10900,1.10904,1.10898,0.0,-0.000018,1.108748,0.000169,1.108916,1.108579,0.0,...,1.108765,1.0,0.0,-1.0,1.108768,1.108816,0.0,0.0,0.0,-1.0


In [45]:
rfm['negitive_returns'] = np.where(rfm['return'] < 0, -1.0, 0.0)
rfm


,Open,High,Low,Volume,return,bollinger_mid_band,bollinger_std,bollinger_upper_band,bollinger_lower_band,bollinger_long,...,slow_close,crossover_long,crossover_short,crossover_signal,SMA5,SMA15,Signal,Entry/Exit,positive_return,negitive_returns
DateTime,,,,,,,,,,,,,,,,,,,,,
2020-02-02 18:01:00,1.10880,1.10890,1.10877,0.0,0.000036,1.108682,0.000129,1.108811,1.108553,0.0,...,1.108732,0.0,-1.0,-1.0,1.108643,1.108832,0.0,0.0,1.0,0.0
2020-02-02 18:02:00,1.10882,1.10891,1.10881,0.0,0.000045,1.108698,0.000132,1.108830,1.108565,0.0,...,1.108737,0.0,-1.0,-1.0,1.108655,1.108826,0.0,0.0,1.0,0.0
2020-02-02 18:03:00,1.10888,1.10898,1.10888,0.0,0.000099,1.108714,0.000146,1.108860,1.108567,0.0,...,1.108746,0.0,-1.0,-1.0,1.108677,1.108823,0.0,0.0,1.0,0.0
2020-02-02 18:04:00,1.10898,1.10902,1.10895,0.0,0.000027,1.108731,0.000160,1.108891,1.108571,0.0,...,1.108756,1.0,0.0,-1.0,1.108704,1.108819,0.0,0.0,1.0,0.0
2020-02-02 18:05:00,1.10900,1.10904,1.10898,0.0,-0.000018,1.108748,0.000169,1.108916,1.108579,0.0,...,1.108765,1.0,0.0,-1.0,1.108768,1.108816,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-20 15:54:00,1.06985,1.06993,1.06936,0.0,-0.000449,1.069219,0.000391,1.069610,1.068828,0.0,...,1.067961,1.0,0.0,1.0,1.069482,1.067756,1.0,0.0,0.0,-1.0
2020-03-20 15:55:00,1.06936,1.07002,1.06936,0.0,0.000365,1.069273,0.000386,1.069660,1.068887,0.0,...,1.068022,1.0,0.0,1.0,1.069488,1.067805,1.0,0.0,1.0,0.0
2020-03-20 15:56:00,1.06968,1.06997,1.06968,0.0,0.000019,1.069318,0.000390,1.069709,1.068928,0.0,...,1.068082,1.0,0.0,1.0,1.069498,1.067858,1.0,0.0,1.0,0.0


In [46]:
# Construct the dependent variable where if daily return is greater than 0, then 1, else, 0.
rfm['positive_return'] = np.where(rfm['return'] > 0, 1.0, 0.0)
rfm

,Open,High,Low,Volume,return,bollinger_mid_band,bollinger_std,bollinger_upper_band,bollinger_lower_band,bollinger_long,...,slow_close,crossover_long,crossover_short,crossover_signal,SMA5,SMA15,Signal,Entry/Exit,positive_return,negitive_returns
DateTime,,,,,,,,,,,,,,,,,,,,,
2020-02-02 18:01:00,1.10880,1.10890,1.10877,0.0,0.000036,1.108682,0.000129,1.108811,1.108553,0.0,...,1.108732,0.0,-1.0,-1.0,1.108643,1.108832,0.0,0.0,1.0,0.0
2020-02-02 18:02:00,1.10882,1.10891,1.10881,0.0,0.000045,1.108698,0.000132,1.108830,1.108565,0.0,...,1.108737,0.0,-1.0,-1.0,1.108655,1.108826,0.0,0.0,1.0,0.0
2020-02-02 18:03:00,1.10888,1.10898,1.10888,0.0,0.000099,1.108714,0.000146,1.108860,1.108567,0.0,...,1.108746,0.0,-1.0,-1.0,1.108677,1.108823,0.0,0.0,1.0,0.0
2020-02-02 18:04:00,1.10898,1.10902,1.10895,0.0,0.000027,1.108731,0.000160,1.108891,1.108571,0.0,...,1.108756,1.0,0.0,-1.0,1.108704,1.108819,0.0,0.0,1.0,0.0
2020-02-02 18:05:00,1.10900,1.10904,1.10898,0.0,-0.000018,1.108748,0.000169,1.108916,1.108579,0.0,...,1.108765,1.0,0.0,-1.0,1.108768,1.108816,0.0,0.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-20 15:54:00,1.06985,1.06993,1.06936,0.0,-0.000449,1.069219,0.000391,1.069610,1.068828,0.0,...,1.067961,1.0,0.0,1.0,1.069482,1.067756,1.0,0.0,0.0,-1.0
2020-03-20 15:55:00,1.06936,1.07002,1.06936,0.0,0.000365,1.069273,0.000386,1.069660,1.068887,0.0,...,1.068022,1.0,0.0,1.0,1.069488,1.067805,1.0,0.0,1.0,0.0
2020-03-20 15:56:00,1.06968,1.06997,1.06968,0.0,0.000019,1.069318,0.000390,1.069709,1.068928,0.0,...,1.068082,1.0,0.0,1.0,1.069498,1.067858,1.0,0.0,1.0,0.0


In [68]:
# Construct training start and end dates
training_start = rfm.index.min().strftime(format= '%Y-%m-%d')
training_end = '2020-02-08'

# Construct testing start and end dates
testing_start =  '2020-03-16'
testing_end = rfm.index.max().strftime(format= '%Y-%m-%d')

# Print training and testing start/end dates
print(f"Training Start: {training_start}")
print(f"Training End: {training_end}")
print(f"Testing Start: {testing_start}")
print(f"Testing End: {testing_end}")

Training Start: 2020-02-02
Training End: 2020-02-08
Testing Start: 2020-03-16
Testing End: 2020-03-20


In [69]:
# Construct the x train and y train datasets
x_train = rfm[x_var_list][training_start:training_end]
y_train = rfm['positive_return'][training_start:training_end]

x_train

,crossover_signal,bollinger_signal
DateTime,,
2020-02-02 18:01:00,-1.0,0.0
2020-02-02 18:02:00,-1.0,0.0
2020-02-02 18:03:00,-1.0,-1.0
2020-02-02 18:04:00,-1.0,-1.0
2020-02-02 18:05:00,-1.0,-1.0
...,...,...
2020-02-07 16:55:00,-1.0,0.0
2020-02-07 16:56:00,-1.0,0.0
2020-02-07 16:57:00,-1.0,0.0


In [70]:
# Construct the x test and y test datasets
x_test = rfm[x_var_list][testing_start:testing_end]
y_test = rfm['positive_return'][testing_start:testing_end]

x_test.tail()

,crossover_signal,bollinger_signal
DateTime,,
2020-03-20 15:54:00,1.0,-1.0
2020-03-20 15:55:00,1.0,-1.0
2020-03-20 15:56:00,1.0,0.0
2020-03-20 15:57:00,1.0,-1.0
2020-03-20 15:58:00,1.0,-1.0


In [71]:
x_test.describe

<bound method NDFrame.describe of                      crossover_signal  bollinger_signal
DateTime                                               
2020-03-16 00:00:00               1.0              -1.0
2020-03-16 00:01:00               1.0              -1.0
2020-03-16 00:02:00               1.0              -1.0
2020-03-16 00:03:00               1.0              -1.0
2020-03-16 00:04:00               1.0              -1.0
...                               ...               ...
2020-03-20 15:54:00               1.0              -1.0
2020-03-20 15:55:00               1.0              -1.0
2020-03-20 15:56:00               1.0               0.0
2020-03-20 15:57:00               1.0              -1.0
2020-03-20 15:58:00               1.0              -1.0

[6719 rows x 2 columns]>

In [72]:
 # Fit a SKLearn linear regression using just the training set (X_train, Y_train):
model = RandomForestClassifier(n_estimators=300, max_depth=100, random_state=42)
model.fit(x_train, y_train)

# Make a prediction of "y" values from the x test dataset
predictions = model.predict(x_test)

# Assemble actual y data (Y_test) with predicted y data (from just above) into two columns in a dataframe:
Results = y_test.to_frame()
Results["Predicted Value"] = predictions
Results

,positive_return,Predicted Value
DateTime,,
2020-03-16 00:00:00,0.0,0.0
2020-03-16 00:01:00,0.0,0.0
2020-03-16 00:02:00,0.0,0.0
2020-03-16 00:03:00,0.0,0.0
2020-03-16 00:04:00,0.0,0.0
...,...,...
2020-03-20 15:54:00,0.0,0.0
2020-03-20 15:55:00,1.0,0.0
2020-03-20 15:56:00,1.0,0.0


In [73]:
print(classification_report(y_train, model.predict(x_train)))

              precision    recall  f1-score   support

         0.0       0.58      1.00      0.74      4139
         1.0       0.00      0.00      0.00      2973

    accuracy                           0.58      7112
   macro avg       0.29      0.50      0.37      7112
weighted avg       0.34      0.58      0.43      7112



In [59]:
from collections import Counter

Counter(y_test)

Counter({0.0: 10930, 1.0: 10237})

In [60]:
Counter(y_train)

Counter({1.0: 12574, 0.0: 16069})

In [61]:
x_train

,crossover_signal,bollinger_signal
DateTime,,
2020-02-02 18:01:00,-1.0,0.0
2020-02-02 18:02:00,-1.0,0.0
2020-02-02 18:03:00,-1.0,-1.0
2020-02-02 18:04:00,-1.0,-1.0
2020-02-02 18:05:00,-1.0,-1.0
...,...,...
2020-02-28 16:54:00,-1.0,0.0
2020-02-28 16:55:00,-1.0,1.0
2020-02-28 16:56:00,-1.0,0.0
